In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

if False:
    import os
    # Turn off GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    gpu = False
else:
    gpu = True

from tensorflow import keras
from copy import deepcopy
import tensorflow as tf
from glob import glob
import numpy as np
import os

gpus = tf.config.experimental.list_physical_devices('GPU')

print(gpus)

if gpu:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024 * 3))])

In [ ]:
from glob import glob

models = sorted(glob("model-saved\\*2023\\*[!.dict]"))
models[:5], len(models)

In [ ]:
from utils import data_utils

DATA_DIR = "utils/split/_split_/_type_"
IMG_SIZE = (256, 256)
train_val_X = sorted(glob(DATA_DIR.replace("_split_", "train-val").replace("_type_", "img") + "\\*"))
train_val_Y = sorted(glob(DATA_DIR.replace("_split_", "train-val").replace("_type_", "mask") + "\\*"))

# test_X = sorted(glob(DATA_DIR.replace("_split_", "test").replace("_type_", "img") + "\\*"))
# test_Y = sorted(glob(DATA_DIR.replace("_split_", "test").replace("_type_", "mask") + "\\*"))


# test_data = data_utils.load_testset(test_X, test_Y, IMAGE_SIZE=IMG_SIZE, BATCH_SIZE=1, REMAP="binary")

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def confusion_matrix(y_true, y_pred):
    # True positive 1 & 1
    TP = np.bitwise_and(
        (y_true == 1), (y_pred == 1)).sum()
    # False positive 0 & 1
    FP = np.bitwise_and(
        (y_true == 0), (y_pred == 1)).sum()
    # True negative 0 & 0
    TN = np.bitwise_and(
        (y_true == 0), (y_pred == 0)).sum()
    # False negative 1 & 0
    FN = np.bitwise_and(
        (y_true == 1), (y_pred == 0)).sum()

    output = np.array([[TN, FP], [FN, TP]])

    return output, output.sum()

def run_confusion_matrix(model, dataset, normalize=False):
    BATCH = IMG = 0
    MASK = 1

    matrix = np.array([[0, 0], [0, 0]])
    total = 0

    for data in dataset:
        y_true = np.reshape(data[MASK][BATCH], (256, 256)) \
            .flatten()
        y_pred = model.predict(data[IMG], verbose=0, workers=100)[BATCH] \
            .astype(np.uint8).flatten()

        matrixPartial, totalPartial = confusion_matrix(y_true, y_pred)

        matrix += matrixPartial
        total += totalPartial

    if normalize:
        matrix = matrix / total
    
    return matrix

def mIOU(y_true, y_pred):
    ulabels = np.unique(y_true)[:-1]

    iou = np.zeros(len(ulabels))

    for k, u in enumerate(ulabels):
        inter = (y_true == u) & (y_pred == u)
        union = (y_true == u) | (y_pred == u)

        iou[k] = inter.sum() / union.sum()

    return iou.mean()

def run_mIOU(model, dataset):
    BATCH = IMG = 0
    MASK = 1
    
    mIOUList = []

    for data in dataset:
        y_true = np.reshape(data[MASK][BATCH], (256, 256)) \
            .flatten()
        y_pred = model.predict(data[IMG], verbose=0, workers=100)[BATCH] \
            .astype(np.uint8).flatten()

        mIOUList.append(mIOU(y_true, y_pred))
    return np.array(mIOUList).round(5)

class plot_distribution():

    def __init__(self, plot_name):
        self.plot_name = plot_name
        self.data = {}

    def add_plot(self, name, data):
        self.data[name] = data

    def show_plot(self, save=False):
        fig, ax = plt.subplots()
        ax.set_title(self.plot_name)
        
        sns.set_context("paper")
        
        ax.set_xlabel("mIOU")
        ax.set_ylabel("mIOU frequency")

        ax = sns.histplot(data=self.data, legend=True, kde=True, 
                          common_norm=False, stat="count", ax=ax,
                          palette=sns.color_palette(n_colors=5))

        if save:
            fig.savefig("plot/"+self.plot_name+".svg", format="svg")
        
        self.data = {}
        plt.show()

In [ ]:
outlier_dict = {}
mIOU_dict = {}
import json
from models import deeplabV3
IMG_SIZE = (256, 256)

for modelPath in models:
    train_val_data = data_utils.load_dataset(train_val_X, train_val_Y, 
                                             IMAGE_SIZE=IMG_SIZE, BATCH_SIZE=1,
                                             REMAP="binary", N_FOLDS=5, SEED=42)
            
    for fold, (trainDataset, valDataset, n_classes) in enumerate(train_val_data):
        # model-saved\\01-03-2023\\deeplabv3+_1_lr_0.001_alpha_0.5_256x256_layer_11_binary_Skip
        modelName = modelPath.split("\\")[-1]

        modelFold = int(modelName.split("_")[1])
        modelLr = modelName.split("_")[3]
        modelAlpha = float(modelName.split("_")[5])
        modelLayer = modelName.split("_")[8]

        if (fold != modelFold):
            continue

        mobileLayers = {"shallowLayer": "block_2_project_BN",
                    "deepLayer": f"block_{modelLayer}_project_BN"}

        model = deeplabV3(imageSize=IMG_SIZE, nClasses=n_classes, alpha=modelAlpha, 
                            withArgmax=True, mobileLayers=mobileLayers)
        model.load_weights(modelPath)

        tempList = run_mIOU(model, valDataset)

        tempDict = {
            "Name":modelName,
            "Mean": np.mean(tempList).round(5),
            "Variance": np.var(tempList).round(5),
            "StandardDeviation": np.std(tempList).round(5),
            "Median": np.median(tempList),
            "IoU_ValSet": tempList
        }

        print(f"{tempDict['Name']} | Mean {tempDict['Mean']} | Variance {tempDict['Variance']} | Standard Deviation {tempDict['StandardDeviation']}")

        try:
            mIOU_dict[f"alpha_{modelAlpha}_layer_{modelLayer}"] += [tempDict]
        except (KeyError):
            mIOU_dict[f"alpha_{modelAlpha}_layer_{modelLayer}"] = [tempDict]

        tempDict = tempList = None

# for key in mIOU_dict.keys():
#     with open(key + '.json', 'w') as fout:
#         json.dump({key:mIOU_dict[key]}, fout)

In [ ]:
outlier_dict

In [ ]:
import json

newDict = {}
for key in mIOU_dict.keys():
    newDict[key] = []
    for dicts in mIOU_dict[key]:
        keys = list(dicts.keys())
        newDict[key].append({
            keys[0]:{
                keys[1]: dicts[keys[1]],
                keys[2]: dicts[keys[2]],
                keys[3]: dicts[keys[3]],
                "IoU" : dicts[keys[0]]}})
        
for key in newDict.keys():
    with open(key + '.json', 'w') as fout:
        json.dump({key:newDict[key]}, fout)

In [ ]:
from glob import glob

files = sorted(glob("*.json"))
files[:3]

In [ ]:
import json

i = 0

metricDict = {}

for file in files:
    with open(file, 'r') as fin:
        # read json with a model cross validation and 
        data = json.load(fin)

        split = list(data.keys())[0]
        
        auxDict = {}

        # remove list of IoU
        for key in data[split].keys():
            # del data[split][key]["mIOU_ValSet"]
            data[split][key]["name"] = key

            fold = "fold " + key.split("_")[1]

            auxDict[fold] = data[split][key]

        metricDict[split] = auxDict

len(metricDict)

In [ ]:
list(metricDict.keys())

In [ ]:
def getOutliers(tempList):
    if not isinstance(tempList, np.array.__class__):
        tempList = np.array(tempList)

    q1, q3 = np.quantile(tempList, [.25, .75])

    outlierLess = q1 - 1.5 * (q3-q1)
    outlierMore = q3 + 1.5 * (q3-q1)

    outlier_index = np.logical_or(tempList <= outlierLess, tempList >= outlierMore)

    outlierList = tempList[outlier_index]

    return outlierList, outlier_index

In [ ]:
from IPython.display import display
import pandas as pd

keys = list(metricDict.keys())

for key in keys:
    key = key.split("_")

    split = f"alpha_{key[1]}_layer_{key[3]}"

    plotOBJ = plot_distribution(split.replace("_", " "))

    dictAux = {}
    print(split)
    for fold in sorted(metricDict[split].keys()):
        mIOUValues = np.array(metricDict[split][fold]["mIOU_ValSet"])

        outlierList, outlier_index = getOutliers(mIOUValues)
        lenOutliers = len(outlierList)
        

        dictAux[fold] = {
            "Mean": np.mean(mIOUValues),
            "StandardDeviation": np.std(mIOUValues),
            "Outliers": lenOutliers}

        # mIOUValues = mIOUValues[~outlier_index]

        plotOBJ.add_plot(fold, mIOUValues)

    df = pd.DataFrame(dictAux).transpose()
    display(df)

    plotOBJ.show_plot(save=True)

In [ ]:
mIOUList = run_mIOU(model, valDataset)

In [ ]:
plot_distribution(mIOU_dict['alpha_0.5_layer_11'][0]['deeplabv3+_1_lr_0.001_alpha_0.5_256x256_layer_11_binary_Skip'])

In [ ]:
matrix = run_confusion_matrix(model, valDataset, normalize=True)
print(matrix)

disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=["False", "True"])
disp.plot()
disp.ax_.set(xlabel='Predicted', ylabel='Groundtruth')
plt.show()

In [ ]:
matrix = run_confusion_matrix(model, test_data, normalize=False)
print(matrix)

disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=["False", "True"])
disp.plot()
disp.ax_.set(xlabel='Predicted', ylabel='Groundtruth')
plt.show()